In [1]:
import pandas as pd

In [104]:
df=pd.read_csv('RFM_ht_data.csv',low_memory = False)

In [105]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [127]:
df.CustomerCode.nunique()

123733

In [106]:
df.shape

(332730, 4)

In [107]:
df.dtypes

InvoiceNo        object
CustomerCode     object
InvoiceDate      object
Amount          float64
dtype: object

In [108]:
df.InvoiceNo=df.InvoiceNo.astype(str)
df.CustomerCode=df.CustomerCode.astype(str)

In [109]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)

In [110]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [111]:
df.InvoiceDate.max()

Timestamp('2020-09-30 00:00:00')

In [112]:
df.InvoiceDate.min()

Timestamp('2020-09-01 00:00:00')

Проведем RFM-анализ

In [113]:
#Последняя дата в датасете, от которой будем считать даты последних покупок
last_date=df.InvoiceDate.max()

In [114]:
df_RFM = df.groupby('CustomerCode', as_index=False)\
    .agg({'InvoiceDate': lambda x: (last_date-x.max()).days, #Recency
         'InvoiceNo': 'count', #Frequency
         'Amount': 'sum'})\
    .rename(columns={'InvoiceDate':'recency',
                    'InvoiceNo':'frequency',
                    'Amount':'monetary'})

In [115]:
df_RFM

,CustomerCode,recency,frequency,monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88
...,...,...,...,...
123728,99099927,10,1,961.10
123729,99099936,0,1,1521.78
123730,99099959,8,2,1444.56
123731,99099963,19,1,3018.91


In [116]:
quantiles=df_RFM.quantile(q=[0.25,0.5,0.75])

In [117]:
quantiles

,recency,frequency,monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [118]:
# Деление на сегменты

def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [140]:
df_RFM ['R'] = df_RFM['recency'].apply(RClass, args =('recency', quantiles))
df_RFM ['F'] = df_RFM['frequency'].apply(FMClass, args =('frequency', quantiles))
df_RFM ['M'] = df_RFM['monetary'].apply(FMClass, args =('monetary', quantiles))

In [141]:
df_RFM['RFM'] = df_RFM.R.map(str)+df_RFM.F.map(str)+df_RFM.M.map(str)

In [142]:
df_RFM

,CustomerCode,recency,frequency,monetary,R,F,M,RFM
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443
...,...,...,...,...,...,...,...,...
123728,99099927,10,1,961.10,3,4,3,343
123729,99099936,0,1,1521.78,1,4,3,143
123730,99099959,8,2,1444.56,2,3,3,233
123731,99099963,19,1,3018.91,4,4,2,442


Какое максимальное кол-во покупок было совершено одним пользователем?

In [143]:
df_RFM.frequency.max()

204

Сколько пользователей попало в сегмент 111?

In [148]:
df_RFM.query('RFM=="111"').agg({'CustomerCode':'count'})

CustomerCode    9705
dtype: int64

Сколько пользователей попало в сегмент 311?

In [145]:
df_RFM.RFM.value_counts()['311']

1609

В каком RFM-сегменте самое большое кол-во пользователей?
В каком RFM-сегменте самое маленькое кол-во пользователей?

In [149]:
df_RFM.RFM.value_counts().sort_values(ascending=False)

444    10624
111     9705
443     6729
344     6593
211     5847
       ...  
424       63
114       60
214       60
314       33
414        2
Name: RFM, Length: 64, dtype: int64